# Extracción de features

In [6]:
# Librerías externas
import os
import numpy as np
import cv2

from skimage.feature import hog


def extract_color_features(colored_img):
    hog_image = hog(colored_img, pixels_per_cell=(8, 8),
                     cells_per_block=(4, 4), transform_sqrt=True, visualize=False, multichannel=True)
    return hog_image.flatten()


print("Extracción de features ------------------------------------------------")

print("Extrayendo features de training...")

# Los cropped son considerando la imagen con y de 0:128, y x de 32:-32.
# Los no cropped son considerando la imagen con y de 0:128.
X_train = []
X_train_cropped = []
X_val = []
X_val_cropped = []
X_test = []
X_test_cropped = []
d_train = []
d_val = []
d_test = []

# Leemos las imagenes de training
directory = "FaceMask166"
for filename in os.listdir(directory):
    if filename.startswith("."):
        continue

    # Obtenemos la clasificación
    classification = int(filename[5:8])

    # Obtenemos la imagen
    filename_path = directory + '/' + filename
    number = int(filename_path[-5])

    # Obtenemos las imagen y la recortamos
    img_colored = cv2.imread(filename_path)
    img_colored_cropped = img_colored[0:128, 32:-32]

    # Obtenemos la imagen flipped
    img_colored_flipped = cv2.imread(filename_path)
    img_colored_flipped = cv2.flip(img_colored_flipped, 1)

    # Obtenemos la imagen flipped y la recortamos
    img_colored_flipped_cropped = img_colored_flipped[0:128, 32:-32]
    img_colored_flipped_cropped = cv2.flip(img_colored_flipped_cropped, 1)

    # Obtenemos las features para no cropped
    features = extract_color_features(img_colored)
    features_flipped = extract_color_features(img_colored_flipped)

    # Obtenemos las features para cropped
    features_cropped = extract_color_features(img_colored_cropped)
    features_flipped_cropped = extract_color_features(img_colored_flipped_cropped)

    # Las guardamos
    if number in {1, 2, 3}:
        X_train.append(np.array(features))
        X_train.append(np.array(features_flipped))
        X_train_cropped.append(np.array(features_cropped))
        X_train_cropped.append(np.array(features_flipped_cropped))

        d_train.append(classification)
        d_train.append(classification)
    elif number == 4:
        X_val.append(np.array(features))
        d_val.append(classification)
    elif number in {5, 6}:
        X_test.append(np.array(features))
        d_test.append(classification)

# Convertimos los datos a arrays
X_train = np.array([np.array(xi) for xi in X_train])
X_test = np.array([np.array(xi) for xi in X_test])
X_val = np.array([np.array(xi) for xi in X_val])
X_train_cropped = np.array([np.array(xi) for xi in X_train])
X_test_cropped = np.array([np.array(xi) for xi in X_test])
X_val_cropped = np.array([np.array(xi) for xi in X_val])
d_train = np.array(d_train).astype(int)
d_test = np.array(d_test).astype(int)
d_val = np.array(d_val).astype(int)

Extracción de features ------------------------------------------------
Extrayendo features de training...


## Clasificación

In [8]:

from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from warnings import simplefilter
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

# Librerías externas
import pickle
from sklearn.metrics import confusion_matrix
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# Ignore warnings
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)


def get_train_test_val(X_train, X_test, X_val, d_train, d_test, d_val, l, augmented=True):
    dic = {'A': 16, 'B': 40, 'C': 100, 'D': 166}
    N = dic[l]
    n = 3
    if augmented:
        n = 6
    
    # Por ejemplo, para 16 datos (N = 16, conjunto A), tendremos:
    #   16 * 6 imágenes de training (6 por persona)
    #   16 * 2 imagenes de training (2 por persona)
    #   16 imagenes de validación (1 por persona)
    return X_train[:n*N], X_test[:2*N], X_val[:N], d_train[:n*N], d_test[:2*N], d_val[:N]

max_acc_test = {'A': 0, 'B': 0, 'C': 0, 'D': 0}
max_acc_val = {'A': 0, 'B': 0, 'C': 0, 'D': 0}
max_acc_val_test = {'A': 0, 'B': 0, 'C': 0, 'D': 0}
for l in ['A', 'B', 'C', 'D']:  # , 'B', 'C', 'D'}:
    print("Resultados para non-cropped")

    # Dividimos los sets de datos
    train, test, val, d_tr, d_te, d_v = get_train_test_val(
        X_train, X_test, X_val, d_train, d_test, d_val, l)

    # Normalizamos
    normalization = MinMaxScaler(feature_range=(0, 1))
    train_norm = normalization.fit_transform(train)
    test_norm = normalization.transform(test)
    val_norm = normalization.transform(val)

    # Generamos el modelo de clasificación
    m = svm.LinearSVC()

    # Lo actualizamos con training
    m.fit(train_norm,  d_tr)

    # Predecimos
    val_pred = m.predict(val_norm)
    test_pred = m.predict(test_norm)
    print("VAL", l, accuracy_score(d_v, val_pred))

    # Guardamos imagen para set validación
    # cm = confusion_matrix(d_v, val_pred)
    # f = sns.heatmap(cm, cmap="Blues")
    # plt.savefig(f"cm_val_{l}.png")
    # plt.clf()#plt.clf()
    print("TEST", l, accuracy_score(d_te, test_pred))
    
    # Guardamos imagen para set training
    # cm = confusion_matrix(d_te, test_pred)
    # f = sns.heatmap(cm, cmap="Blues")
    # plt.savefig(f"cm_test_{l}.png")
    # plt.clf()

ModuleNotFoundError: No module named 'model'